In [11]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

client = MongoClient(os.getenv("MONGO_DB_CONNECTION_STRING"))
db = client["vms"]
 
invitesCollection = db["invites"]
groupInvitesCollection = db["groupinvites"]
groupParkingReservationsCollection = db["groupParkingReservations"]
parkingReservationCollection = db["parkingreservations"]
trayCollection = db["trays"]
# groupInvitesCollection.delete_many({})
# groupParkingReservationsCollection.delete_many({})


print(invitesCollection.find_one({})['inviteDate'])
{ '$and': [ { "inviteState": { '$ne': "inActive" } }, { "inviteState": { '$ne': "cancelled" } } ] }
numVis = invitesCollection.count_documents({ '$and': [ { "inviteState": { '$ne': "inActive" } }, { "inviteState": { '$ne': "cancelled" } } ] })
print(numVis)

2016-01-01


OperationFailure: unknown operator: $and, full error: {'ok': 0.0, 'errmsg': 'unknown operator: $and', 'code': 2, 'codeName': 'BadValue', '$clusterTime': {'clusterTime': Timestamp(1661235163, 2), 'signature': {'hash': b'<\r\x1f\x13@/\\m@b\xc4\n\xc4\x8dNJ\x13gK\xdb', 'keyId': 7086437743361785861}}, 'operationTime': Timestamp(1661235163, 2)}

In [13]:
parkingReservationCollection.aggregate([
    {
        '$group': {
            '_id': '$reservationDate', 
            'numParkings': {
                '$sum': 1
            }
        }
    }, {
        '$out': 'groupparkingreservations'
    }
    ])

In [7]:
groupInvites.aggregate([
  {
    "$lookup": {
      "from": "parkingreservations",
      "localField": "_id",
      "foreignField": "reservationDate",
      "as": "parkingDetails"
    }
  },
  {
    "$unwind": "$parkingDetails"
  },
  {
    "$project": {
      "_id": {
        "date": "$inviteDate"
      },
      "numVisitors": {
        "$sum": 1
      },
      "numParking": {
         "$size": "$parkingDetails"
      }
    }
  }, 
  {
    '$out': 'groupParkingReservations'
  }
])


OperationFailure: PlanExecutor error during aggregation :: caused by :: The argument to $size must be an array, but was of type: object, full error: {'ok': 0.0, 'errmsg': 'PlanExecutor error during aggregation :: caused by :: The argument to $size must be an array, but was of type: object', 'code': 17124, 'codeName': 'Location17124', '$clusterTime': {'clusterTime': Timestamp(1660681030, 3), 'signature': {'hash': b')\x8c\x1a\r\xe5\\\x0f\x80c\xb1\xbb@\x11=ah\xba\xd3\x14\xf0', 'keyId': 7086437743361785861}}, 'operationTime': Timestamp(1660681030, 1)}